# Aufgaben zu Big Data File Formats

Folgende Aufgaben haben zum Ziel mit den verschiedenen Dateiformaten vertraut zu werden und insbesondere die speziellen Eigenschaften und Funktionen der Formate zu verstehen

### CSV and JSON
Klassische Datei Formate für Datenverarbeitung  
**Typische Eigenschaften:** Einfache Struktur, human-readable, Zeilenformat

### Avro, ORC, Parquet
Big Data optimierte Formate um schnell große Datenmengen zu lesen und zu schreiben  
**Typische Eigenschaften:** teilbar in kleine Dateien (splittable), komprimierbar (compressible), überspringbar (skippable), selbsterklärend (self describing with schema), Schema erweiterbar (Schema Evolution), Schema erzwingend (Schema Enforcment), Filter Pushdown

### Delta, Iceberg, Hudi
Erweiterte Big Data Formate um die ACID und Tracing Eigenschaften einer klassichen SQL Datenbank zu erfüllen  
**Typische Eigenschaften:** Erweiterung um zusätzliche Metadaten und spezielle Treiber zum lesen/schreiben, Time Travel Funktion, Merge und Update Funktionen, Audit Log Funktionalitäten

###  Import Python Modules
Hier werden alle benötigten Libraries für dieses Lab heruntergeladen.


In [1]:
from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession
from pyspark.sql import SQLContext
from pyspark.sql.types import *
import pyspark.sql.functions as f

from delta import *

import datetime
from datetime import datetime, timedelta
import time
import json
import csv

import boto3
from botocore.client import Config

# use 95% of the screen for jupyter cell
from IPython.display import display, HTML
display(HTML("<style>.container {width:100% !important; }<style>"))

### Launch Spark Jupyter and Configuration

#### Configure a Spark session for Kubernetes cluster with S3 support
### CLUSTER MANAGER
- set the Kubernetes master URL as Cluster Manager(“k8s://https://” is NOT a typo, this is how Spark knows the “provider” type)

### KUBERNETES
- set the namespace that will be used for running the driver and executor pods
- set the docker image from which the Worker/Exectutor pods are created
- set the Kubernetes service account name and provide the authentication details for the service account (required to create worker pods)

### SPARK
- set the driver host and the driver port (find name of the driver service with 'kubectl get services' or in the helm chart configuration)
- enable Delta Lake, Iceberg, and Hudi support by setting the spark.sql.extensions
- configure Hive catalog for Iceberg
- enable S3 connector
- set the number of worker pods, their memory and cores (HINT: number of possible tasks = cores * executores)

### SPARK SESSION
- create the Spark session using the SparkSession.builder object
- get the Spark context from the created session and set the log level to "ERROR".


In [2]:
appName="jupyter-file-formats"

conf = SparkConf()

# CLUSTER MANAGER

conf.setMaster("k8s://https://kubernetes.default.svc.cluster.local:443")

# CONFIGURE KUBERNETES

conf.set("spark.kubernetes.namespace","frontend")
conf.set("spark.kubernetes.container.image", "thinkportgmbh/workshops:spark-3.3.2")
conf.set("spark.kubernetes.container.image.pullPolicy", "Always")

conf.set("spark.kubernetes.authenticate.driver.serviceAccountName", "spark")
conf.set("spark.kubernetes.authenticate.caCertFile", "/var/run/secrets/kubernetes.io/serviceaccount/ca.crt")
conf.set("spark.kubernetes.authenticate.oauthTokenFile", "/var/run/secrets/kubernetes.io/serviceaccount/token")

# CONFIGURE SPARK

conf.set("spark.sql.session.timeZone", "Europe/Berlin")
conf.set("spark.driver.host", "jupyter-spark-driver.frontend.svc.cluster.local")
conf.set("spark.driver.port", "29413")

conf.set("spark.jars", "/opt/spark/jars/spark-avro_2.12-3.3.2.jar")
conf.set("spark.driver.extraClassPath","/opt/spark/jars/spark-avro_2.12-3.3.2.jar")
conf.set("spark.executor.extraClassPath","/opt/spark/jars/spark-avro_2.12-3.3.2.jar")

conf.set("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension, org.apache.iceberg.spark.extensions.IcebergSparkSessionExtensions, org.apache.spark.sql.hudi.HoodieSparkSessionExtension")

######## Hive als Metastore einbinden
#conf.set("hive.metastore.uris", "thrift://hive-metastore.hive.svc.cluster.local:9083") 

######## Iceberg configs
conf.set("spark.sql.catalog.spark_catalog", "org.apache.iceberg.spark.SparkSessionCatalog")
conf.set("spark.sql.catalog.ice","org.apache.iceberg.spark.SparkCatalog") 
conf.set("spark.sql.catalog.ice.type","hive") 
conf.set("spark.sql.catalog.ice.uri","thrift://hive-metastore.hive.svc.cluster.local:9083") 

####### Hudi configs
conf.set("spark.serializer", "org.apache.spark.serializer.KryoSerializer")

# CONFIGURE S3 CONNECTOR
conf.set("spark.hadoop.fs.s3a.endpoint", "minio.minio.svc.cluster.local:9000")
conf.set("spark.hadoop.fs.s3a.access.key", "trainadm")
conf.set("spark.hadoop.fs.s3a.secret.key", "train@thinkport")
conf.set("spark.hadoop.fs.s3a.path.style.access", "true")
conf.set("spark.hadoop.fs.s3.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem")
conf.set("spark.hadoop.fs.s3a.aws.credentials.provider", "org.apache.hadoop.fs.s3a.SimpleAWSCredentialsProvider")
conf.set("spark.hadoop.fs.s3a.connection.ssl.enabled", "false")

# CONFIGURE WORKER (Customize based on workload)

conf.set("spark.executor.instances", "2")
conf.set("spark.executor.memory", "1G")
conf.set("spark.executor.cores", "2")

# SPARK SESSION

spark = SparkSession\
    .builder\
    .config(conf=conf) \
    .config('spark.sql.session.timeZone', 'Europe/Berlin') \
    .appName(appName)\
    .enableHiveSupport() \
    .getOrCreate()


sc=spark.sparkContext
sc.setLogLevel("ERROR")

# get the configuration object to check all the configurations the session was startet with
for entry in sc.getConf().getAll():
        if entry[0] in ["spark.app.name","spark.kubernetes.namespace","spark.executor.memory","spark.executor.cores","spark.driver.host","spark.master","spark.sql.extensions"]:
            print(entry[0],"=",entry[1])

23/07/21 07:51:43 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


spark.kubernetes.namespace = frontend
spark.sql.extensions = io.delta.sql.DeltaSparkSessionExtension, org.apache.iceberg.spark.extensions.IcebergSparkSessionExtensions, org.apache.spark.sql.hudi.HoodieSparkSessionExtension
spark.master = k8s://https://kubernetes.default.svc.cluster.local:443
spark.executor.memory = 1G
spark.executor.cores = 2
spark.driver.host = jupyter-spark-driver.frontend.svc.cluster.local
spark.app.name = jupyter-file-formats


### Configure Boto3 
for simple s3 operations

In [3]:
# Hilfsfunktionen um mit einfachen Befehlen auf s3 zu arbeiten 
# WICHTIG: Falls das bucket s3://fileformats noch nicht existiert muss dieses über das Terminal erst erzeugt werden
# Command: s3 mb s3://fileformats

# Bucket, muss zuerst in Minio oder via Terminal Befehl erstellt werden
bucket = "fileformats"
bucket_path="s3://"+bucket

options = {
    'endpoint_url': 'http://minio.minio.svc.cluster.local:9000',
    'aws_access_key_id': 'trainadm',
    'aws_secret_access_key': 'train@thinkport',
    'config': Config(signature_version='s3v4'),
    'verify': False}

s3_resource = boto3.resource('s3', **options)  
s3_client = boto3.client('s3', **options)

# show files on s3 bucket/prefix
def ls(bucket,prefix):
    '''List objects from bucket/prefix'''
    try:
        for obj in s3_resource.Bucket(bucket).objects.filter(Prefix=prefix):
            print(obj.key)
    except Exception as e: 
        print(e)
    
# show file content in files
def cat(bucket,prefix,binary=False):
    '''Show content of one or several files with same prefix/wildcard'''
    try:
        for obj in s3_resource.Bucket(bucket).objects.filter(Prefix=prefix):
            print("File:",obj.key)
            print("----------------------")
            if binary==True:
                print(obj.get()['Body'].read())
            else: 
                print(obj.get()['Body'].read().decode())
            print("######################")
    except Exception as e: 
        print(e)

# delete files from bucket
def rm(bucket,prefix):
    '''Delete everything from bucket/prefix'''
    for object in s3_resource.Bucket(bucket).objects.filter(Prefix=prefix):
        print(object.key)
        s3_client.delete_object(Bucket=bucket, Key=object.key)
    print(f"Deleted files from {bucket}/{prefix}*")


In [53]:
#rm(bucket,'delta_scd2')

In [12]:
# show everything in bucket
ls(bucket,"")
print("All Folders #############################")
# show folder
ls(bucket,"csv")
print("CSV Details #############################")
# show subfolder
ls(bucket,"delta/_delta_log/")
print("Delta Details #############################")
print("")
# show content of one or several files with same prefix/wildcard
cat(bucket,'csv/part')

iceberg/data/00000-7912-ace0e8c1-fb4a-4812-bdf3-6d4c0ad58a06-00001.parquet
iceberg/data/00000-7920-9dd85db3-402a-4657-856c-f0e8526d3256-00001.parquet
iceberg/data/00001-7913-df5b342a-250c-4439-a721-bf25810804f3-00001.parquet
iceberg/data/00002-7914-112c31b7-0460-4fed-b4c9-8b9a5f1dbfd5-00001.parquet
iceberg/metadata/00000-d4590277-2b88-433d-aa9f-8ba595501aef.metadata.json
iceberg/metadata/00001-fb578df1-af09-4ce8-854a-d66befc4519b.metadata.json
iceberg/metadata/00002-becfce86-8178-4223-8c6d-84baaf06b31b.metadata.json
iceberg/metadata/0a07116f-9521-4d1d-bc4a-3a729535f621-m0.avro
iceberg/metadata/4d380afb-46c9-4d4c-a877-135bf0ec57a9-m0.avro
iceberg/metadata/snap-2832191032484601030-1-0a07116f-9521-4d1d-bc4a-3a729535f621.avro
iceberg/metadata/snap-868766759905674702-1-4d380afb-46c9-4d4c-a877-135bf0ec57a9.avro
All Folders #############################
CSV Details #############################
Delta Details #############################



### Create sample data

In [13]:
# initial Daten
account_data1 = [
    (1,"alex","2019-01-01",1000),
    (2,"alex","2019-02-01",1500),
    (3,"alex","2019-03-01",1700),
    (4,"maria","2020-01-01",5000)
    ]

# Datensatz mit einem Update und einer neuen Zeile
account_data2 = [
    (1,"alex","2019-03-01",3300),
    (2,"peter","2021-01-01",100)
    ]

# Datensatz mit neuer Zeile und neuer Spalte
account_data3 = [
    (1,"otto","2019-10-01",4444,"neue Spalte 1")
]

# Datensatz mit neuer Zeile und neuer Spalte
account_data4 = [
    (5,"markus","2019-09-01",555)
]

schema = ["id","account","dt_transaction","balance"]
schema3 = ["id","account","dt_transaction","balance","new"]

df1 = spark.createDataFrame(data=account_data1, schema = schema).withColumn("dt_transaction",f.col("dt_transaction").cast("date")).repartition(3)
df2 = spark.createDataFrame(data=account_data2, schema = schema).withColumn("dt_transaction",f.col("dt_transaction").cast("date")).repartition(2)
df3 = spark.createDataFrame(data=account_data3, schema = schema3).withColumn("dt_transaction",f.col("dt_transaction").cast("date")).repartition(1)
df4 = spark.createDataFrame(data=account_data4, schema = schema).withColumn("dt_transaction",f.col("dt_transaction").cast("date")).withColumn("id",f.col("id").cast("string")).repartition(1)


print("++ create new dataframe and show schema and data")
print("################################################")

# df1.printSchema()
print("++ start data")
df1.show(truncate=False)
print("++ update row and add row")
df2.show(truncate=False)
print("++ add new column")
df3.show(truncate=False)
print("++ add new row with wrong schema (id)")
df4.show(truncate=False)
df1.printSchema()
df4.printSchema()

++ create new dataframe and show schema and data
################################################
++ start data


+---+-------+--------------+-------+
|id |account|dt_transaction|balance|
+---+-------+--------------+-------+
|4  |maria  |2020-01-01    |5000   |
|1  |alex   |2019-01-01    |1000   |
|2  |alex   |2019-02-01    |1500   |
|3  |alex   |2019-03-01    |1700   |
+---+-------+--------------+-------+

++ update row and add row
+---+-------+--------------+-------+
|id |account|dt_transaction|balance|
+---+-------+--------------+-------+
|2  |peter  |2021-01-01    |100    |
|1  |alex   |2019-03-01    |3300   |
+---+-------+--------------+-------+

++ add new column
+---+-------+--------------+-------+-------------+
|id |account|dt_transaction|balance|new          |
+---+-------+--------------+-------+-------------+
|1  |otto   |2019-10-01    |4444   |neue Spalte 1|
+---+-------+--------------+-------+-------------+

++ add new row with wrong schema (id)
+---+-------+--------------+-------+
|id |account|dt_transaction|balance|
+---+-------+--------------+-------+
|5  |markus |2019-09-01    |555

<hr style="height: 3px; background: gray;">

## CSV

### Aufgabe:
Schreibe die Daten als CSV mit der Overwrite und Append Funktion

1. Datenset 1 als csv schreiben (.format("csv") und Pfad= .save(f"s3://{bucket}/csv"))
2. Dateien und Inhalt anzeigen, vestehen was da passiert ist
3. Daten wieder einlese und checken ob es ein Schema und Spaltennamen erhalten wurden
4. Datenset 3 anfügen mit weiterer Spalte anfügen (append)
5. Daten wieder einlesen und checken was mit der neuen Spalte passiert


In [14]:
print("Number of Partitions:", df1.rdd.getNumPartitions())

# Schreibe Datenset 1 als CSV Datei
write_csv=(df1
           .write
           .format("csv")
           .mode("overwrite") # append
           .save(f"s3://{bucket}/csv")
          )

Number of Partitions: 3


In [15]:
# Anzeigen der Dateien im Bucket/Prefix
ls(bucket,"csv")

csv/_SUCCESS
csv/part-00000-40f490ff-8cfe-404b-808e-5c2372751fb9-c000.csv
csv/part-00001-40f490ff-8cfe-404b-808e-5c2372751fb9-c000.csv
csv/part-00002-40f490ff-8cfe-404b-808e-5c2372751fb9-c000.csv


In [16]:
# Anzeigen der Inhalte jeder Datei im Bucket/Prefix
cat(bucket,"csv/part")

File: csv/part-00000-40f490ff-8cfe-404b-808e-5c2372751fb9-c000.csv
----------------------
1,alex,2019-01-01,1000
4,maria,2020-01-01,5000

######################
File: csv/part-00001-40f490ff-8cfe-404b-808e-5c2372751fb9-c000.csv
----------------------
2,alex,2019-02-01,1500

######################
File: csv/part-00002-40f490ff-8cfe-404b-808e-5c2372751fb9-c000.csv
----------------------
3,alex,2019-03-01,1700

######################


In [17]:
# lese die csv Datei wieder ein und prüfe das Schema
read_csv=spark.read.format("csv").load(f"s3://{bucket}/csv")

read_csv.printSchema()
read_csv.show()

root
 |-- _c0: string (nullable = true)
 |-- _c1: string (nullable = true)
 |-- _c2: string (nullable = true)
 |-- _c3: string (nullable = true)



+---+-----+----------+----+
|_c0|  _c1|       _c2| _c3|
+---+-----+----------+----+
|  1| alex|2019-01-01|1000|
|  4|maria|2020-01-01|5000|
|  2| alex|2019-02-01|1500|
|  3| alex|2019-03-01|1700|
+---+-----+----------+----+



In [18]:
# schreibe Datenset 3 (neue Spalte) in die gleiche Tabelle dazu
write_csv=(df3
           .write
           .format("csv")
           .mode("append")
           .save(f"s3://{bucket}/csv")
          )

In [19]:
# Anzeigen der Inhalte jeder Datei im Bucket/Prefix
ls(bucket,"csv")

csv/_SUCCESS
csv/part-00000-40f490ff-8cfe-404b-808e-5c2372751fb9-c000.csv
csv/part-00000-4ad5fd72-1534-41a4-8f87-f5eba25d4a99-c000.csv
csv/part-00001-40f490ff-8cfe-404b-808e-5c2372751fb9-c000.csv
csv/part-00002-40f490ff-8cfe-404b-808e-5c2372751fb9-c000.csv


In [20]:
# Anzeigen der Inhalte jeder Datei im Bucket/Prefix
cat(bucket,"csv/part")

File: csv/part-00000-40f490ff-8cfe-404b-808e-5c2372751fb9-c000.csv
----------------------
1,alex,2019-01-01,1000
4,maria,2020-01-01,5000

######################
File: csv/part-00000-4ad5fd72-1534-41a4-8f87-f5eba25d4a99-c000.csv
----------------------
1,otto,2019-10-01,4444,neue Spalte 1

######################
File: csv/part-00001-40f490ff-8cfe-404b-808e-5c2372751fb9-c000.csv
----------------------
2,alex,2019-02-01,1500

######################
File: csv/part-00002-40f490ff-8cfe-404b-808e-5c2372751fb9-c000.csv
----------------------
3,alex,2019-03-01,1700

######################


In [21]:
# und lese alles nochmal ein um zu schauen ob die neue Spalte richtig erkannt wurde
read_csv=spark.read.format("csv").load(f"s3://{bucket}/csv")

read_csv.printSchema()
read_csv.show()

root
 |-- _c0: string (nullable = true)
 |-- _c1: string (nullable = true)
 |-- _c2: string (nullable = true)
 |-- _c3: string (nullable = true)

+---+-----+----------+----+
|_c0|  _c1|       _c2| _c3|
+---+-----+----------+----+
|  1| alex|2019-01-01|1000|
|  4|maria|2020-01-01|5000|
|  1| otto|2019-10-01|4444|
|  2| alex|2019-02-01|1500|
|  3| alex|2019-03-01|1700|
+---+-----+----------+----+



#### Erkenntnisse CSV
* In wieviele Dateien wird das Datenset aufgeteilt und warum?
* Bleibt das Schema erhalten (Selbsterklärend)
* Können neue Spalten angefügt werden (Schema Evolution)

<hr style="height: 3px; background: gray;">

## JSON

### Aufgabe:
Wiederhole die gleichen Schritte mit dem JSON Format und schaue wie sich hier Schema und neue Spalten verhalten

1. Datenset 1 als json schreiben (.format("json") und Pfad= .save(f"s3://{bucket}/json"))
2. Dateien und Inhalt anzeigen, vestehen was da passiert ist
3. Daten wieder einlese und checken ob es ein Schema und Spaltennamen gibt
4. Datenset 3 anfügen (append)
5. Daten wieder einlesen und checken was mit der neuen Spalte passiert


In [22]:
print("Number of Partitions:", df1.rdd.getNumPartitions())

# Schreibe Datenset 1 als JSON Datei
write_json=(df1
           .write
           .format("json")
           .mode("overwrite") # append
           .save(f"s3://{bucket}/json")
          )

Number of Partitions: 3


<details style="border: 1px solid #aaa; border-radius: 4px; padding: 0.5em 0.5em 0; background-color:#F5F5F5" class="solution" >
<summary style="margin: -0.5em -0.5em 0; padding: 0.5em;"></summary>
Lösung:<br>
<code>write_json=(df1
           .write
           .format("json")
           .mode("overwrite") # append
           .save(f"s3://{bucket}/json")
          )</code>
</details>


In [23]:
ls(bucket,"json")

json/_SUCCESS
json/part-00000-0db0c2eb-c91f-482e-a6bf-22043ff52fb6-c000.json
json/part-00001-0db0c2eb-c91f-482e-a6bf-22043ff52fb6-c000.json
json/part-00002-0db0c2eb-c91f-482e-a6bf-22043ff52fb6-c000.json


In [24]:
cat(bucket,"json/part")

File: json/part-00000-0db0c2eb-c91f-482e-a6bf-22043ff52fb6-c000.json
----------------------
{"id":1,"account":"alex","dt_transaction":"2019-01-01","balance":1000}
{"id":4,"account":"maria","dt_transaction":"2020-01-01","balance":5000}

######################
File: json/part-00001-0db0c2eb-c91f-482e-a6bf-22043ff52fb6-c000.json
----------------------
{"id":2,"account":"alex","dt_transaction":"2019-02-01","balance":1500}

######################
File: json/part-00002-0db0c2eb-c91f-482e-a6bf-22043ff52fb6-c000.json
----------------------
{"id":3,"account":"alex","dt_transaction":"2019-03-01","balance":1700}

######################


In [25]:
# Daten wieder einlese und checken ob es ein Schema und Spaltennamen gibt
read_json=spark.read.format("json").load(f"s3://{bucket}/json")

read_json.printSchema()
read_json.show()


root
 |-- account: string (nullable = true)
 |-- balance: long (nullable = true)
 |-- dt_transaction: string (nullable = true)
 |-- id: long (nullable = true)

+-------+-------+--------------+---+
|account|balance|dt_transaction| id|
+-------+-------+--------------+---+
|   alex|   1000|    2019-01-01|  1|
|  maria|   5000|    2020-01-01|  4|
|   alex|   1500|    2019-02-01|  2|
|   alex|   1700|    2019-03-01|  3|
+-------+-------+--------------+---+



In [26]:
# schreibe Datenset 3 (neue Spalte) in die gleiche Tabelle dazu (!! append NOT overwrite)

write_json=(df3
           .write
           .format("json")
           .mode("append") # append
           .save(f"s3://{bucket}/json")
          )

<details style="border: 1px solid #aaa; border-radius: 4px; padding: 0.5em 0.5em 0; background-color:#F5F5F5" class="solution" >
<summary style="margin: -0.5em -0.5em 0; padding: 0.5em;"></summary>
Lösung:<br>
<code>
    <p>
    write_json=(df3
           .write
           .format("json")
           .mode("append") # append
           .save(f"s3://{bucket}/json")
          )
    </p>
</code>
</details>

In [27]:
# alles nochmal einlesen und schauen ob die neue Spalte und die Schemas richtig erkannt wurden

read_json=spark.read.format("json").load(f"s3://{bucket}/json")

read_json.printSchema()
read_json.show()


root
 |-- account: string (nullable = true)
 |-- balance: long (nullable = true)
 |-- dt_transaction: string (nullable = true)
 |-- id: long (nullable = true)
 |-- new: string (nullable = true)

+-------+-------+--------------+---+-------------+
|account|balance|dt_transaction| id|          new|
+-------+-------+--------------+---+-------------+
|   alex|   1000|    2019-01-01|  1|         null|
|  maria|   5000|    2020-01-01|  4|         null|
|   otto|   4444|    2019-10-01|  1|neue Spalte 1|
|   alex|   1500|    2019-02-01|  2|         null|
|   alex|   1700|    2019-03-01|  3|         null|
+-------+-------+--------------+---+-------------+



<details style="border: 1px solid #aaa; border-radius: 4px; padding: 0.5em 0.5em 0; background-color:#F5F5F5" class="solution" >
<summary style="margin: -0.5em -0.5em 0; padding: 0.5em;"></summary>
Lösung:<br>
<code>
read_json=spark.read.format("json").load(f"s3://{bucket}/json")
read_json.printSchema()
read_json.show()
    </code>
</details>

#### Erkenntnisse JSON
* Bleibt das Schema erhalten (Selbsterklärend)?
* Können neue Spalten angefügt werden (Schema Evolution)?

## AVRO
Avro ist ein Zeilenformat was für das schnelle Schreiben im Streaming Kontext optimiert ist.
Avro ist selbsterklärend, hat ein Schema und unterstützt Schema Evolution

### Aufgabe:
Wiederhole die gleichen Schritte mit dem AVRO Format und schaue wie sich hier Schema und neue Spalten verhalten

1. Datenset 1 als avro schreiben (.format("avro") und Pfad= .save(f"s3://{bucket}/avro"))
2. Dateien und Inhalt anzeigen, vestehen was da passiert ist
3. Metadaten in Datei identifizieren
3. Daten wieder einlese und checken ob es ein Schema und Spaltennamen gibt
4. Schema Evolutiuon: Datenset 3 anfügen mit neuer Spalte anfügen
5. Daten wieder einlesen und checken was mit der neuen Spalte passiert
6. Schema Enforcement: Datentyp in bestehender Spalte ändern und schauen ob und wie dies gehandhabt wird

In [28]:
print("Number of Partitions:", df1.rdd.getNumPartitions())
# Schreibe Datenset 1 als AVRO Datei
write_avro=(df1
           .write
           .format("avro")
           .mode("overwrite") # append
           .save(f"s3://{bucket}/avro")
          )

Number of Partitions: 3


<details style="border: 1px solid #aaa; border-radius: 4px; padding: 0.5em 0.5em 0; background-color:#F5F5F5" class="solution" >
<summary style="margin: -0.5em -0.5em 0; padding: 0.5em;"></summary>
Lösung:<br>
<code>write_avro=(df1
           .write
           .format("avro")
           .mode("overwrite") # append
           .save(f"s3://{bucket}/avro")
          )</code>
</details>

In [29]:
# sind die Daten auf s3 angekommen
ls(bucket,"avro")

avro/_SUCCESS
avro/part-00000-afd8998c-ebe8-4d8d-b740-8e8e6f6fd1de-c000.avro
avro/part-00001-afd8998c-ebe8-4d8d-b740-8e8e6f6fd1de-c000.avro
avro/part-00002-afd8998c-ebe8-4d8d-b740-8e8e6f6fd1de-c000.avro


In [30]:
# Finde in der Darstellung der Datei die Metadaten und die eigentlichen Daten
# Da Avro ein Binärformat ist muss hier in cat die Flag auf True gesetzt werden
cat(bucket,"avro/part",True)

File: avro/part-00000-afd8998c-ebe8-4d8d-b740-8e8e6f6fd1de-c000.avro
----------------------
b'Obj\x01\x06\x16avro.schema\xfa\x03{"type":"record","name":"topLevelRecord","fields":[{"name":"id","type":["long","null"]},{"name":"account","type":["string","null"]},{"name":"dt_transaction","type":[{"type":"int","logicalType":"date"},"null"]},{"name":"balance","type":["long","null"]}]}0org.apache.spark.version\n3.3.2\x14avro.codec\x0csnappy\x00R\xfd<\x14\x967pE2\xce\x10n\x03\x18\xee[\x04J\x1fx\x00\x02\x00\x08alex\x00\xd2\x97\x02\x00\xd0\x0f\x00\x08\x00\nmaria\x00\xac\x9d\x02\x00\x90N\x9d\xdfwVR\xfd<\x14\x967pE2\xce\x10n\x03\x18\xee['
######################
File: avro/part-00001-afd8998c-ebe8-4d8d-b740-8e8e6f6fd1de-c000.avro
----------------------
b'Obj\x01\x06\x16avro.schema\xfa\x03{"type":"record","name":"topLevelRecord","fields":[{"name":"id","type":["long","null"]},{"name":"account","type":["string","null"]},{"name":"dt_transaction","type":[{"type":"int","logicalType":"date"},"null"]},{"na

In [31]:
read_avro=spark.read.format("avro").load(f"s3://{bucket}/avro")


read_avro.printSchema()
read_avro.show()

root
 |-- id: long (nullable = true)
 |-- account: string (nullable = true)
 |-- dt_transaction: date (nullable = true)
 |-- balance: long (nullable = true)

+---+-------+--------------+-------+
| id|account|dt_transaction|balance|
+---+-------+--------------+-------+
|  1|   alex|    2019-01-01|   1000|
|  4|  maria|    2020-01-01|   5000|
|  2|   alex|    2019-02-01|   1500|
|  3|   alex|    2019-03-01|   1700|
+---+-------+--------------+-------+



### Avro: Schema Evolution

In [32]:
# schreibe Datenset 3 (neue Spalte) in die gleiche Tabelle dazu (!! append NOT overwrite)

write_avro=(df3
           .write
           .format("avro")
           .mode("append") # append
           .save(f"s3://{bucket}/avro")
          )

<details style="border: 1px solid #aaa; border-radius: 4px; padding: 0.5em 0.5em 0; background-color:#F5F5F5" class="solution" >
<summary style="margin: -0.5em -0.5em 0; padding: 0.5em;"></summary>
Lösung:<br>
<code>write_avro=(df3
           .write
           .format("avro")
           .mode("append") # append
           .save(f"s3://{bucket}/avro")
          )</code>
</details>

In [33]:
# alles nochmal einlesen und schauen ob die neue Spalte und die Schemas richtig erkannt wurden
# wiederholt sich langsam gell?

read_avro=spark.read.format("avro").load(f"s3://{bucket}/avro")


read_avro.printSchema()
read_avro.show()

root
 |-- id: long (nullable = true)
 |-- account: string (nullable = true)
 |-- dt_transaction: date (nullable = true)
 |-- balance: long (nullable = true)
 |-- new: string (nullable = true)

+---+-------+--------------+-------+-------------+
| id|account|dt_transaction|balance|          new|
+---+-------+--------------+-------+-------------+
|  1|   otto|    2019-10-01|   4444|neue Spalte 1|
|  1|   alex|    2019-01-01|   1000|         null|
|  4|  maria|    2020-01-01|   5000|         null|
|  2|   alex|    2019-02-01|   1500|         null|
|  3|   alex|    2019-03-01|   1700|         null|
+---+-------+--------------+-------+-------------+



<details style="border: 1px solid #aaa; border-radius: 4px; padding: 0.5em 0.5em 0; background-color:#F5F5F5" class="solution" >
<summary style="margin: -0.5em -0.5em 0; padding: 0.5em;"></summary>
Lösung AVRO wieder einlesen:<br>
<code>
read_avro=spark.read.format("avro").load(f"s3://{bucket}/json")
read_avro.printSchema()
read_avro.show()
    </code>
</details>

### Avro: Schema Enforcement

In [34]:
# Füge eine Zeile (df2) zu der AVRO Tabelle hinzu aber ändere den Datentyp für die id von long zu string
print("Schema vorher:")
df2.printSchema()


df2a=(df2
      # nur die Zeile Peter aus df2
      .where(f.col("account")=="peter")
      # ID als string statt als long
      .withColumn("id", f.col("id").cast("int"))
     )

print("Schema nachher:")
df2a.printSchema()


write_avro=(df2a
            .write
            .format("avro")
            .mode("append")
            .save(f"s3://{bucket}/avro")
           )


Schema vorher:
root
 |-- id: long (nullable = true)
 |-- account: string (nullable = true)
 |-- dt_transaction: date (nullable = true)
 |-- balance: long (nullable = true)

Schema nachher:
root
 |-- id: integer (nullable = true)
 |-- account: string (nullable = true)
 |-- dt_transaction: date (nullable = true)
 |-- balance: long (nullable = true)



In [35]:
# probiere das Verzeichnis jetzt mit verschiedenen Datentypen einzulesen
read_avro=(spark
               .read
               .format("avro")
               .load(f"s3://{bucket}/avro"))
read_avro.printSchema()

try:
    read_avro.show()
except Exception as error:
    error_str=str(error)
    search="Cannot convert Avro field 'id' to SQL field"
    print("++ filter in Spark Error:")
    print(error_str[error_str.find(search)-51:error_str.find(search)+118])
    print("Schema enforcement on read")

root
 |-- id: integer (nullable = true)
 |-- account: string (nullable = true)
 |-- dt_transaction: date (nullable = true)
 |-- balance: long (nullable = true)



23/07/21 07:57:12 ERROR TaskSetManager: Task 0 in stage 62.0 failed 4 times; aborting job
++ filter in Spark Error:
apache.spark.sql.avro.IncompatibleSchemaException: Cannot convert Avro field 'id' to SQL field 'id' because schema is incompatible (avroType = "long", sqlType = INT)
	at
Schema enforcement on read


#### Erkenntnisse AVRO
* Werden Spaltennamen erhalten? 
* Gibt es ein Schema?
* Schema Evolution: Kann das Schema erweitert werden, also eine neue Spalte angefügt werden?
* Schema Enforcement on write: Kann eine Spalte mit falschem Datetyp einfach beim schreiben hinzugefügt werden? 
* Schema Enforcement on read: Kann ein Verzeichnis mit mehreren Avro Dateien bei der eine Spalte ein anderes Schema hat gelesen werden?

<hr style="height: 3px; background: gray;">

## Parquet

### Aufgabe:
Wiederhole die gleichen Schritte mit dem PARQUET Format und schaue wie sich hier Schema und neue Spalten verhalten

1. Datenset 1 als parquet schreiben (.format("parquet") und Pfad= .save(f"s3://{bucket}/parquet"))
2. Dateien und Inhalt anzeigen, vestehen was da passiert ist
3. Metadaten in Datei identifizieren
3. Daten wieder einlese und checken ob es ein Schema und Spaltennamen gibt
4. Schema Evolutiuon: Datenset 3 anfügen mit neuer Spalte anfügen
5. Daten wieder einlesen und checken was mit der neuen Spalte passiert
6. Partion & Pushdown Filter: Execution Plan für verschiedene Filter anzeigen
6. Schema Enforcement: Datentyp in bestehender Spalte ändern und schauen ob und wie dies gehandhabt wird

In [36]:
print("Number of Partitions:", df1.rdd.getNumPartitions())

write_parquet=(df1
           .write
           # Fachliche Partitionierung beim Schreiben
           .partitionBy("account")
           .format("parquet")
           .mode("overwrite")
           .save(f"s3://{bucket}/parquet")
          )


Number of Partitions: 3


In [37]:
# sind die Daten auf s3 angekommen
ls(bucket,"parquet")

parquet/_SUCCESS
parquet/account=alex/part-00000-8ac618ae-a8d0-412c-8be3-a918e3f9b0d2.c000.snappy.parquet
parquet/account=alex/part-00001-8ac618ae-a8d0-412c-8be3-a918e3f9b0d2.c000.snappy.parquet
parquet/account=alex/part-00002-8ac618ae-a8d0-412c-8be3-a918e3f9b0d2.c000.snappy.parquet
parquet/account=maria/part-00000-8ac618ae-a8d0-412c-8be3-a918e3f9b0d2.c000.snappy.parquet


In [38]:
# schaue eine Datei im Detail an und finde die Metadaten
cat(bucket,"parquet",True)

File: parquet/_SUCCESS
----------------------
b''
######################
File: parquet/account=alex/part-00000-8ac618ae-a8d0-412c-8be3-a918e3f9b0d2.c000.snappy.parquet
----------------------
b'PAR1\x15\x00\x15\x1c\x15 \x15\xfe\xc8\xd0\xb6\x0b\x1c\x15\x02\x15\x00\x15\x06\x15\x08\x00\x00\x0e4\x02\x00\x00\x00\x03\x01\x01\x00\x00\x00\x00\x00\x00\x00\x15\x00\x15\x14\x15\x18\x15\xbd\x9c\xff\xa7\x07\x1c\x15\x02\x15\x00\x15\x06\x15\x08\x00\x00\n$\x02\x00\x00\x00\x03\x01\xe9E\x00\x00\x15\x00\x15\x1c\x15 \x15\xaf\xa4\x96\x9e\x01\x1c\x15\x02\x15\x00\x15\x06\x15\x08\x00\x00\x0e4\x02\x00\x00\x00\x03\x01\xe8\x03\x00\x00\x00\x00\x00\x00\x19\x11\x02\x19\x18\x08\x01\x00\x00\x00\x00\x00\x00\x00\x19\x18\x08\x01\x00\x00\x00\x00\x00\x00\x00\x15\x02\x19\x16\x00\x00\x19\x11\x02\x19\x18\x04\xe9E\x00\x00\x19\x18\x04\xe9E\x00\x00\x15\x02\x19\x16\x00\x00\x19\x11\x02\x19\x18\x08\xe8\x03\x00\x00\x00\x00\x00\x00\x19\x18\x08\xe8\x03\x00\x00\x00\x00\x00\x00\x15\x02\x19\x16\x00\x00\x19\x1c\x16\x08\x15N\x16\x00\x00\x00

<details style="border: 1px solid #aaa; border-radius: 4px; padding: 0.5em 0.5em 0; background-color:#F5F5F5" class="solution" >
<summary style="margin: -0.5em -0.5em 0; padding: 0.5em;"></summary>
Lösung PARQUET Metadaten anzeigen:<br>
<code>
cat(bucket,"parquet/account=maria",True)
</code>
</details>

### Parquet: Filter Pushdown
Da das Parquet Format spalten basiert ist und für jede Spalte Metadaten vorhällt, können Programme die diese Dateien einlesen vor der Serialisierung (dem kompletten Einlesen in den Arbeitsspeicher) erst die Header scannen und entscheiden welche Dateien tatsächlich benötigt werden.  
Dies nennt man Attribut oder Filter Pushdown.  
Spark kann außerdem, wenn die Daten in Partition im Format `PartitionKey=value` diese automatisch erkennen und wenn ein Filter auf die Partition gelegt ist nur diesen Unterordner einlesen.  

**Aufgabe:** Untersuche den Execution Plan der Filter Operation 

In [39]:
# Parquet Datei mit PartitionFilter laden
read_parquet=(spark
              .read.format("parquet")
              .load(f"s3://{bucket}/parquet")
              # Filter auf die Spalte über die partitioniert wurde
              .filter(f.col("account")=="alex")
             )

# Parquet mit normalem Filter laden
read_parquet2=(spark
              .read.format("parquet")
              .load(f"s3://{bucket}/parquet")
              # Filter auf die Spalte über eine normale Spalte
              .filter(f.col("balance")>1500)
              .select("balance")
             )

# Parquet mit normalem Filter laden
read_parquet3=(spark
              .read.format("parquet")
              .load(f"s3://{bucket}/parquet")
              # Filter auf die Spalte über die partitioniert wurde
              .filter(f.col("account")=="alex")
              # Filter auf die Spalte über eine normale Spalte
              .filter(f.col("balance")>1500)
              .select("account","balance")
             )

# Anzeigen des physischen Execution Plans um zu sehen welche Filter ins Dateisystem bzw. in die Parquet Datei gepusht werden
print("Partition Filter")
print("#######################")
read_parquet.explain("simple")
print("Pushdow Filter")
print("#######################")
read_parquet2.explain("simple")
print("All Filter")
print("#######################")
read_parquet3.explain("simple")



Partition Filter
#######################
== Physical Plan ==
*(1) ColumnarToRow
+- FileScan parquet [id#579L,dt_transaction#580,balance#581L,account#582] Batched: true, DataFilters: [], Format: Parquet, Location: InMemoryFileIndex(1 paths)[s3://fileformats/parquet], PartitionFilters: [isnotnull(account#582), (account#582 = alex)], PushedFilters: [], ReadSchema: struct<id:bigint,dt_transaction:date,balance:bigint>


Pushdow Filter
#######################
== Physical Plan ==
*(1) Project [balance#590L]
+- *(1) Filter (isnotnull(balance#590L) AND (balance#590L > 1500))
   +- *(1) ColumnarToRow
      +- FileScan parquet [balance#590L,account#591] Batched: true, DataFilters: [isnotnull(balance#590L), (balance#590L > 1500)], Format: Parquet, Location: InMemoryFileIndex(1 paths)[s3://fileformats/parquet], PartitionFilters: [], PushedFilters: [IsNotNull(balance), GreaterThan(balance,1500)], ReadSchema: struct<balance:bigint>


All Filter
#######################
== Physical Plan ==
*(1) Project

### Parquet: Schema Evolution
Schema Evolution ermöglicht es das Schema der Tabelle zu erweitern.  
Der Spark Parquet reader bietet verschiedenen Möglichkeiten mit Schemaerweiterungen umzugehen  
https://spark.apache.org/docs/latest/sql-data-sources-parquet.html#schema-merging

**Aufgabe:** Lese die Daten so ein, dass das Schema korrekt erweitert wird

In [40]:
# Zeile mit neuer Spalte anfügen
write_parquet=(df3
           .write
           .format("parquet")
           .mode("append") # append
           # schreibe ohne zu Partitionieren direkt in ein neues Unterverzeichnis
           .save(f"s3://{bucket}/parquet/account=otto")
          )

In [41]:
# einlesen mit der mergeSchema Option
read_parquet=(spark
              .read.format("parquet")
              # setzte die mergeSchema auf true/false um den Unterschied beim Einlesen zu sehen
              # Vegleiche: https://spark.apache.org/docs/latest/sql-data-sources-parquet.html#schema-merging
              #.option("mergeSchema", "false")
              .load(f"s3://{bucket}/parquet")
             )

read_parquet.printSchema()
read_parquet.show()

root
 |-- id: long (nullable = true)
 |-- dt_transaction: date (nullable = true)
 |-- balance: long (nullable = true)
 |-- account: string (nullable = true)



+---+--------------+-------+-------+
| id|dt_transaction|balance|account|
+---+--------------+-------+-------+
|  1|    2019-10-01|   4444|   otto|
|  1|    2019-01-01|   1000|   alex|
|  2|    2019-02-01|   1500|   alex|
|  3|    2019-03-01|   1700|   alex|
|  4|    2020-01-01|   5000|  maria|
+---+--------------+-------+-------+



### Parquet: Schema Enforcement
Schema Enforcement sorgt dafür, dass in ein bestehendes Schema keine Daten mit falschen Typen geschrieben werden können

In [42]:
# Datensatz mit falschem Datentyp anfügen
df2a=(df2.where(f.col("account")=="peter").withColumn("id", f.col("id").cast("string")))


# Zeile mit falschem Typ anfügen
write_parquet=(df2a
           .write
           .partitionBy("account")
           .format("parquet")
           .mode("append") 
           .save(f"s3://{bucket}/parquet")
          )

In [43]:
# einlesen mit der mergeSchema Option
read_parquet=(spark
              .read.format("parquet")
              # setzte die mergeSchema auf true/false um den Unterschied beim Einlesen zu sehen
              .option("mergeSchema", "false")
              .load(f"s3://{bucket}/parquet")
             )

read_parquet.printSchema()


try:
    read_parquet.show()
except Exception as error:
    error_str=str(error)
    #print(error_str)
    search="Parquet column cannot be converted in file s3"
    print("++ filter in Spark Error:")
    print(error_str[error_str.find(search)-51:error_str.find(search)+198])
    print("++ this is schema enforcement on read")

root
 |-- id: long (nullable = true)
 |-- dt_transaction: date (nullable = true)
 |-- balance: long (nullable = true)
 |-- account: string (nullable = true)



23/07/21 07:57:37 ERROR TaskSetManager: Task 1 in stage 80.0 failed 4 times; aborting job
++ filter in Spark Error:
pache.spark.sql.execution.QueryExecutionException: Parquet column cannot be converted in file s3://fileformats/parquet/account=peter/part-00000-b86c829d-5c70-41c9-9153-d6d74c540e84.c000.snappy.parquet. Column: [id], Expected: bigint, Found: BINARY
	
++ this is schema enforcement on read


#### Erkenntnisse Parquet
* Sind Parquet Dateien selbsterklärend (haben ein Spalten und Typenschema )
* Partitioning and Partion Discovery: werden die Daten in Verzeichnisse geschriebe und wieder als Partitionen erkannt?
* Schema Evolution: Kann das Schema erweitert werden, also eine neue Spalte angefügt werden?
* Schema Enforcement on write: Kann eine Spalte mit falschem Datetyp einfach beim schreiben hinzugefügt werden? 
* Schema Enforcement on read: Kann ein Verzeichnis mit mehreren Parquet Dateien bei der eine Spalte ein anderes Schema hat gelesen werden?

<hr style="height: 3px; background: gray;">

# Delta
Das Delta Format fügt Parquet Dateien einen zusätzlichen Layer an Metadatan hinzu und erfüllt mit dem entsprechenden Treiber alle ACID Eigenschaften einer Datenbank und mehr. 

A = **Atomic** heißt alle Datenänderungen werden wie eine einzige Operation verarbeitet. Dies bedeutet, dass entweder alle Änderungen durchgeführt werden oder keine. Wenn das schreiben also mitten drin Fehlschlägt werden alle Daten dieser Schreiboperation die bereits geschrieben wurden wieder entfernt, bzw. nicht als erfolgreich geschrieben markiert.  
T = **Consistency** bedeutet, wenn eine Transaktion beginnt und wenn eine Transaktion endet, befinden sich die Daten in einem konsistenten Zustand.  
I = **Isolation** und bedeutet, dass der Übergangszustand einer Transaktion für andere Transaktionen nicht sichtbar ist. Dies führt dazu, dass Transaktionen, die gleichzeitig ablaufen, sich nicht gegenseitig beeinflussen oder blockieren.  
D =**Durability** heißt, das die Datenänderungen nach erfolgreich abgeschlossener Transaktion erhalten bleiben und werden nicht rückgängig gemacht werden, selbst wenn ein Systemausfall auftritt.  

**Time Travel** bei Delta bedeutet, dass jede Datenänderung als eigene Version aufgezeichnet wird und jederzeit zu einer alten Version zurück gegegangen werden kann.

### Aufgabe:
Wiederhole die gleichen Schritte mit dem DELTA Format und schaue wie sich hier Schema und neue Spalten verhalten

1. Datenset 1 als DELTA schreiben (.format("delta") und Pfad= .save(f"s3://{bucket}/delta"))
2. Dateien und Inhalt anzeigen, vestehen was da passiert ist
3. Metadaten und Deltalog in Datei verstehen
3. Daten wieder einlese und checken ob es ein Schema und Spaltennamen gibt
4. Schema Evolutiuon: Datenset 3 anfügen mit neuer Spalte anfügen
5. Daten wieder einlesen und checken was mit der neuen Spalte passiert
6. Partion & Pushdown Filter: Execution Plan für verschiedene Filter anzeigen
6. Schema Enforcement: Datentyp in bestehender Spalte ändern und schauen ob und wie dies gehandhabt wird

In [44]:
# Schreibe die Daten df1 als Delta Datei in den Pfad bucket/delta
write_delta=(df1
           .write
           .format("delta")
           .option("overwriteSchema", "true")
           .mode("overwrite") 
           .save(f"s3://{bucket}/delta")
          )

In [45]:
ls(bucket,"delta")

delta/_delta_log/00000000000000000000.json
delta/part-00000-a8a8ace7-c5fb-4f8a-9ddc-d0ebfc4e1181-c000.snappy.parquet
delta/part-00001-84e3dcfd-61b2-4cf1-8207-6d438e6439f7-c000.snappy.parquet
delta/part-00002-c92c8bd2-8907-474d-b48c-420e6b8ef06e-c000.snappy.parquet


In [46]:
ls(bucket,"delta/_delta_log")

delta/_delta_log/00000000000000000000.json


In [47]:
# untersuche die Dateien des Deltalogs und versuche zu verstehen wie die Versionierung und Schema Validierung damit funktioniert
cat(bucket,"delta/_delta_log")

File: delta/_delta_log/00000000000000000000.json
----------------------
{"commitInfo":{"timestamp":1689926289159,"operation":"WRITE","operationParameters":{"mode":"Overwrite","partitionBy":"[]"},"isolationLevel":"Serializable","isBlindAppend":false,"operationMetrics":{"numFiles":"3","numOutputRows":"4","numOutputBytes":"3727"},"engineInfo":"Apache-Spark/3.3.2 Delta-Lake/2.3.0","txnId":"599b6d5d-abe5-4072-979b-fbc8afbc62d5"}}
{"protocol":{"minReaderVersion":1,"minWriterVersion":2}}
{"metaData":{"id":"a0ab151a-0f86-49ee-b19c-a01418b1daa4","format":{"provider":"parquet","options":{}},"schemaString":"{\"type\":\"struct\",\"fields\":[{\"name\":\"id\",\"type\":\"long\",\"nullable\":true,\"metadata\":{}},{\"name\":\"account\",\"type\":\"string\",\"nullable\":true,\"metadata\":{}},{\"name\":\"dt_transaction\",\"type\":\"date\",\"nullable\":true,\"metadata\":{}},{\"name\":\"balance\",\"type\":\"long\",\"nullable\":true,\"metadata\":{}}]}","partitionColumns":[],"configuration":{},"createdTime":1

In [48]:
# lese die gerade geschriebenen Delta Tabelle wieder ein und zeige sie an, überprüfe das Schema
read_delta=(spark.read.format("delta").load(f"s3://{bucket}/delta"))

read_delta.printSchema()
read_delta.show()


root
 |-- id: long (nullable = true)
 |-- account: string (nullable = true)
 |-- dt_transaction: date (nullable = true)
 |-- balance: long (nullable = true)



+---+-------+--------------+-------+
| id|account|dt_transaction|balance|
+---+-------+--------------+-------+
|  1|   alex|    2019-01-01|   1000|
|  4|  maria|    2020-01-01|   5000|
|  2|   alex|    2019-02-01|   1500|
|  3|   alex|    2019-03-01|   1700|
+---+-------+--------------+-------+



<details style="border: 1px solid #aaa; border-radius: 4px; padding: 0.5em 0.5em 0; background-color:#F5F5F5" class="solution" >
<summary style="margin: -0.5em -0.5em 0; padding: 0.5em;"></summary>
<code>
read_delta=spark.read.format("delta").load(f"s3://{bucket}/delta")
read_delta.show()
</code>
</details>

### Delta: Schema Evolution
**Aufgabe:** Verstehe die Option *mergeSchema* on write

In [49]:
# Zeile mit zusätzlicher Spalte anfügen (df3)
write_delta=(df3
           .write
           .format("delta")
           # Bei Delta kann bein Schreiben gesetzt werden ob die Tabelle erweitert werden soll oder nicht, Default ist false. 
           # Führe den Code zuerst ohne diese Option aus und schaue das Ergebnis, an 
           .option("mergeSchema", "true")
           .mode("append") # append
           .save(f"s3://{bucket}/delta")
          )


# überprüfe ob die neue Spalte korrekt angefügt wurde
read_delta=spark.read.format("delta").load(f"s3://{bucket}/delta")
read_delta.show()

+---+-------+--------------+-------+-------------+
| id|account|dt_transaction|balance|          new|
+---+-------+--------------+-------+-------------+
|  1|   otto|    2019-10-01|   4444|neue Spalte 1|
|  1|   alex|    2019-01-01|   1000|         null|
|  4|  maria|    2020-01-01|   5000|         null|
|  2|   alex|    2019-02-01|   1500|         null|
|  3|   alex|    2019-03-01|   1700|         null|
+---+-------+--------------+-------+-------------+



In [50]:
cat(bucket,"delta/_delta_log")

File: delta/_delta_log/00000000000000000000.json
----------------------
{"commitInfo":{"timestamp":1689926289159,"operation":"WRITE","operationParameters":{"mode":"Overwrite","partitionBy":"[]"},"isolationLevel":"Serializable","isBlindAppend":false,"operationMetrics":{"numFiles":"3","numOutputRows":"4","numOutputBytes":"3727"},"engineInfo":"Apache-Spark/3.3.2 Delta-Lake/2.3.0","txnId":"599b6d5d-abe5-4072-979b-fbc8afbc62d5"}}
{"protocol":{"minReaderVersion":1,"minWriterVersion":2}}
{"metaData":{"id":"a0ab151a-0f86-49ee-b19c-a01418b1daa4","format":{"provider":"parquet","options":{}},"schemaString":"{\"type\":\"struct\",\"fields\":[{\"name\":\"id\",\"type\":\"long\",\"nullable\":true,\"metadata\":{}},{\"name\":\"account\",\"type\":\"string\",\"nullable\":true,\"metadata\":{}},{\"name\":\"dt_transaction\",\"type\":\"date\",\"nullable\":true,\"metadata\":{}},{\"name\":\"balance\",\"type\":\"long\",\"nullable\":true,\"metadata\":{}}]}","partitionColumns":[],"configuration":{},"createdTime":1

### Delta: Schema Enforcement
Schema Enforcement bedeutet soll garantieren, dass keine Daten mit falschen Datentyp der Tabelle abgefügt werden  

**Aufgabe:** verstehe die Option *mergeSchema* im Kontext von Datentyp Änderungen bei bestehenden Spalten, funktioniert das?

In [51]:
# füge eine Zeile mit falschen Datetyp für eine bestehenden Spalte an
write_delta=(df2
           # Eine Zeile aus dem df2 filtern
           .where(f.col("account")=="peter")
           # Bestehenden Spaltentyp ändern
           .withColumn("id", f.col("id").cast("string"))
           .write
           .format("delta")
           # Bei Delta kann beim Schreiben gesetzt werden ob die Tabelle erweitert werden kann oder nicht, Default ist false
           .option("mergeSchema", "true")
           .mode("overwrite") 
           .save(f"s3://{bucket}/delta")
          )

AnalysisException: Failed to merge fields 'id' and 'id'. Failed to merge incompatible data types LongType and StringType

#### Ergebnis: geht nicht! 
Delta garantiert Schema Enforcement on write. Die Option "mergeSchema", "true" gilt nur für Schema Erweiterung, also dem Anfügen neuer Spalten, 
nicht aber dem ändern von bestehenden Datetypen

### Delta: Schema Replacement
Delta bieten die Möglichkeit das Schema einer Tabelle zu ändern, also z.B. eine bestehende Spalte umzubenennen und deren Datentyp zu ändern.   
Dokumentation mit Beispielen: https://docs.delta.io/latest/delta-batch.html#replace-table-schema  

**Aufgabe:** Ändere in der bestehenden Tabelle den Datentyp der Spalte `id` auf `string` und den Spaltennamen `new` zu `comment`. 

In [ ]:
# Update das Schema der bestehenen Delta option("overwriteSchema", "true")
write_delta=(df2
           # Eine Zeile aus dem df2 filtern
           .where(f.col("account")=="peter")
           # Bestehenden Spaltentyp ändern
           .withColumn("id", f.col("id").cast("string"))
           .write
           .format("delta") 
           .option("overwriteSchema", "true") # wichtig Overwrite Schema geht nur mit dem Mode overwrite, also ein neues Schema für die neuen Dateien, es geht nicht mit einem Append alle Dateien zu altern
           .mode("overwrite") 
           .save(f"s3://{bucket}/delta")
          )

In [ ]:
read_delta=spark.read.format("delta").load(f"s3://{bucket}/delta")
read_delta.printSchema()
read_delta.show()

### Delta: History und Metadaten
Im Delta Log werde alle Transaktionen mit zahlreichen Metadaten gespeichert. Das Spark Modul, der Treiber, um diese Daten auszulesen bietet zahlreiche Möglichkeiten diese Daten zu analysieren

**Aufgabe:** Lese den History Log ein und verstehe was in den einzelnen Attributen steht. Treffe eine Auswahl der interessanten Informationen

In [ ]:
# Erzeuge ein DeltaTable Objekt was alle Zusatzeigenschaften von Delta bereitstellt
deltaTable = DeltaTable.forPath(spark, f"s3://{bucket}/delta")

# Historie aus den Delta Logs erzeugen
fullHistoryDF = deltaTable.history() 

# Alle verfügbaren Spalten anzeigen
fullHistoryDF.printSchema()

In [ ]:
# Wähle die wichtigen Felder aus der Historie aus und zeige sie an
fullHistoryDF.select("version","readVersion","timestamp","userId","operation","operationParameters","operationMetrics","userMetadata").show(truncate=False)

# Löse die genested Spalten in eigene Spalten auf und zeige folgende Attribute in eigenen Spalten `mode`, `numFiles` und `numOutputRows` 
fullHistoryDF.select("version","readVersion","timestamp","operation","operationParameters.mode","operationMetrics.numFiles","operationMetrics.numOutputRows").show(truncate=False)

In [ ]:
# Erkunde die Metadaten aus der Funktion deltaTable.detail()

deltaTable.detail().select("format","name","location","createdAt","lastModified","numFiles","tableFeatures").show(truncate=False)

<details style="border: 1px solid #aaa; border-radius: 4px; padding: 0.5em 0.5em 0; background-color:#F5F5F5" class="solution" >
<summary style="margin: -0.5em -0.5em 0; padding: 0.5em;"></summary>
<code>
deltaTable = DeltaTable.forPath(spark, f"s3://{bucket}/delta")
detailsDF = deltaTable.detail()
detailsDF.show()
detailsDF.select("location","numFiles","tableFeatures").show(truncate=False)
</code>
</details>

### Delta: Time Travel
Die Time Travel Funktion von Delta ermöglicht es den Zustand der Datentabelle zu einem bestimmten Zeitpunkt in der Vergangenheit wiederherzustellen oder Änderungen zu verfolgen.   
Time Travel ermöglicht es, vorherige Versionen der Daten abzufragen und historische Analysen durchzuführen, ohne auf separate Backups oder Snapshots angewiesen zu sein.

**Aufgabe:** lese verschiedenen Datenstände nach Versionsnummer oder Timestap ein.  
Weitere Informationen hierzu finden sich in https://delta.io/blog/2023-02-01-delta-lake-time-travel/

In [ ]:
# read by version
spark.read.format("delta").option("versionAsOf", "8").load(f"s3://{bucket}/delta").show()

In [ ]:
# read by timestamp
spark.read.format("delta").option("timestampAsOf", "2023-07-20 15:55:00").load(f"s3://{bucket}/delta").show()

### Delta: Roleback
Delta bietet die Möglichkeiten direkt auf den Datenstand einer bestimmten Version oder eines Zeitpunktes zurück zu gehen

**Aufgabe:** Setzte die aktuelle Version der deltaTable auf den Anfangszusatand des df1 mit vier Zeilen zurück

In [ ]:
# Verwende die DeltaTable Funktion restoreToVersion(0) oder restoreToTimestamp("yyyy-mm-dd")
deltaTable2 = DeltaTable.forPath(spark, f"s3://{bucket}/delta")

deltaTable2.restoreToVersion(0).show()

In [ ]:
deltaTable2.toDF().show()

<details style="border: 1px solid #aaa; border-radius: 4px; padding: 0.5em 0.5em 0; background-color:#F5F5F5" class="solution" >
<summary style="margin: -0.5em -0.5em 0; padding: 0.5em;">&#8964 Hilfreiche Befehle</summary>
<code>
deltaTable2.toDF().show()

deltaTable2.restoreToVersion(0)
#restoreToTimestamp
#isDeltaTable
deltaTable2.toDF().show()
spark.read.format("delta").load(f"s3://{bucket}/delta").show()

</code>
</details>

### Delta: Merge (Upsert)
Delta bietet die Möglichkeiten auf bestehenden Dateien ein Daten Upsert durchzuführen.   
Ubsert oder Merge bedeutet zu prüfen ob es die neue Zeile für einen bestimmten Schlüssel schon in den Daten gibt und wenn ja diese zu updaten und wenn nein sie neu hinzuzufügen

**Aufgabe:** Merge den Datensatz df2 mit dem der Änderung der `balance` für `Alex` und überprüfe ob das Update korrekt war

In [ ]:
# Spalte anfügen, da merge nur funktioniert wenn das Schema stimmt
df2a=df2

print("++ Datensatz der auf bestehende Daten upserted/merged werden soll")
df2a.show()
print("++ Bestehender Datensatz auf s3")
deltaTable2.toDF().show()

In [ ]:
# Verwendung der merge Funktion (es gibt auch eine update() oder delete() Funktion)
dt3=(deltaTable2.alias("oldData")
      .merge(df2a.alias("newData"),
            "oldData.account = newData.account AND oldData.dt_transaction = newData.dt_transaction")
            .whenMatchedUpdateAll()
            .whenNotMatchedInsertAll()
      .execute()
    )

deltaTable2.toDF().sort(f.col("account"),f.col("dt_transaction")).show()

### Delta: SCD2 Historisierung
Mit der Merge Funktion von Delta lässt sich z.B. einfach eine Datenhistorisierung nach dem SCD2 Prinzip realisieren  
(Slowly Changing Dimensions Typ 2, https://de.wikipedia.org/wiki/Slowly_Changing_Dimensions) 
SCD2 ermöglicht es Änderungen einzelner Datensätze zu tracken. Typischerweise werden neben dem Änderungsdatum auch noch Metadaten wie Grund und User mit gespeichert

In [54]:
# Initial Dataframe erstellen
df_initial = df1


init_load_ts = "2023-06-01"

# Die drei SCD2 Spalten valid_from, valid_to und is_current hinzufügen
df_initial = (df_initial
              .drop("id")
              .withColumn('valid_from', f.lit(init_load_ts).cast("timestamp"))
              .withColumn('valid_to', f.lit(None).cast("timestamp"))
              .withColumn('is_current', f.lit(True))
             )

# in neue Deltatabelle=Datei schreiben
df_initial.write.format('delta').mode("overwrite").option("overwriteSchema", "true").save(f"s3://{bucket}/delta_scd2")
dTable_initial = DeltaTable.forPath(spark, f"s3://{bucket}/delta_scd2")
df_initial=dTable_initial.toDF()

df_initial.show(truncate=False)

+-------+--------------+-------+-------------------+--------+----------+
|account|dt_transaction|balance|valid_from         |valid_to|is_current|
+-------+--------------+-------+-------------------+--------+----------+
|maria  |2020-01-01    |5000   |2023-06-01 00:00:00|null    |true      |
|alex   |2019-01-01    |1000   |2023-06-01 00:00:00|null    |true      |
|alex   |2019-03-01    |1700   |2023-06-01 00:00:00|null    |true      |
|alex   |2019-02-01    |1500   |2023-06-01 00:00:00|null    |true      |
+-------+--------------+-------+-------------------+--------+----------+



In [58]:
 # Update Dataframe vorbereiten
df_current = (df2
          # Duplikat hinzufügen
          .unionByName(df1.where(f.col("account")=="maria"))
          # komplexität reduzieren
          .drop("id")
          # der neue Datensatz gilt ab jetzt
          .withColumn('valid_from', f.lit(f.current_timestamp()))
          # und gilt wieder bis in die unendlichkeit
          .withColumn('valid_to', f.lit(None).cast("timestamp"))
          .withColumn('is_current', f.lit(True))
          # Hilfsspalte um zu markieren was mit der Zeile gemacht werden soll, new insert, update existing, delete row. Erstmal alles auf update
          .withColumn('action', f.lit('insert'))
         ).cache()

df_current.show(truncate=False)

+-------+--------------+-------+--------------------------+--------+----------+------+
|account|dt_transaction|balance|valid_from                |valid_to|is_current|action|
+-------+--------------+-------+--------------------------+--------+----------+------+
|peter  |2021-01-01    |100    |2023-07-21 10:16:35.890389|null    |true      |insert|
|alex   |2019-03-01    |3300   |2023-07-21 10:16:35.890389|null    |true      |insert|
|maria  |2020-01-01    |5000   |2023-07-21 10:16:35.890389|null    |true      |insert|
+-------+--------------+-------+--------------------------+--------+----------+------+



In [59]:
# Finde Zeilen die es schon gibt und die einen neuen Wert für Balance haben. 
# Erzeuge den Update Eintrag um diese Zeile zu deaktivieren
df_rows_to_deactivate = (df_initial.alias("initial")
                .join(df_current.alias("current"), 
                      (df_initial.account == df_current.account) & 
                      (df_initial.dt_transaction == df_current.dt_transaction) 
                      , how='inner'
                     )
                .where((f.col("initial.balance")!=f.col("current.balance")) & (f.col("initial.valid_to").isNull()))
                .select("initial.*")
                .withColumn("action",f.lit("deactivate"))
                .withColumn("valid_to",f.current_timestamp())
                .withColumn("is_current",f.lit(False))
               )

print("++ Zeilen die es schon gibt aber deren Wert sich geändert haben werden deaktiviert")
df_rows_to_deactivate.show(truncate=False)

# Finde Zeilen die es schon gibt und wo sich kein Wert geändert hat, sie bleiben erhalten
df_rows_do_not_touch= (df_initial.alias("initial")
                .join(df_current.alias("current"), 
                      (df_initial.account == df_current.account) & 
                      (df_initial.dt_transaction == df_current.dt_transaction) 
                      , how='inner'
                     )
                .where((f.col("initial.balance")==f.col("current.balance")))
                .select("initial.*")
                .withColumn("action",f.lit("duplicate"))
               )


print("Zeilen die es schon gibt, wo sich aber nichts ändert müssen von der Ubsert Liste entfernt werden")
df_rows_do_not_touch.show(truncate=False)


print("Jetzt werden die neuen Zeilen mit den Deaktivierten Zeilen vereint und dann die Duplicate entfernt")
df_rows_for_upsert=(df_current
                # füge als extra Zeile die deaktiverten Einträge hinzu
                .union(df_rows_to_deactivate)
                # bereits existierende Zeilen entfernen
                .join(df_rows_do_not_touch,
                      (df_current.account == df_rows_do_not_touch.account) & 
                      (df_current.dt_transaction == df_rows_do_not_touch.dt_transaction) &
                      (df_current.balance == df_rows_do_not_touch.balance),
                      how="left_anti"
                     )
                 )


df_rows_for_upsert.show(truncate=False)


++ Zeilen die es schon gibt aber deren Wert sich geändert haben werden deaktiviert
+-------+--------------+-------+-------------------+--------------------------+----------+----------+
|account|dt_transaction|balance|valid_from         |valid_to                  |is_current|action    |
+-------+--------------+-------+-------------------+--------------------------+----------+----------+
|alex   |2019-03-01    |1700   |2023-06-01 00:00:00|2023-07-21 10:16:37.421142|false     |deactivate|
+-------+--------------+-------+-------------------+--------------------------+----------+----------+

Zeilen die es schon gibt, wo sich aber nichts ändert müssen von der Ubsert Liste entfernt werden
+-------+--------------+-------+-------------------+--------+----------+---------+
|account|dt_transaction|balance|valid_from         |valid_to|is_current|action   |
+-------+--------------+-------+-------------------+--------+----------+---------+
|maria  |2020-01-01    |5000   |2023-06-01 00:00:00|null    

+-------+--------------+-------+--------------------------+--------------------------+----------+----------+
|account|dt_transaction|balance|valid_from                |valid_to                  |is_current|action    |
+-------+--------------+-------+--------------------------+--------------------------+----------+----------+
|peter  |2021-01-01    |100    |2023-07-21 10:16:42.551346|null                      |true      |insert    |
|alex   |2019-03-01    |3300   |2023-07-21 10:16:42.551346|null                      |true      |insert    |
|alex   |2019-03-01    |1700   |2023-06-01 00:00:00       |2023-07-21 10:16:42.551346|false     |deactivate|
+-------+--------------+-------+--------------------------+--------------------------+----------+----------+



+-------+--------------+-------+-------------------+--------+----------+
|account|dt_transaction|balance|         valid_from|valid_to|is_current|
+-------+--------------+-------+-------------------+--------+----------+
|  maria|    2020-01-01|   5000|2023-06-01 00:00:00|    null|      true|
|   alex|    2019-01-01|   1000|2023-06-01 00:00:00|    null|      true|
|   alex|    2019-03-01|   1700|2023-06-01 00:00:00|    null|      true|
|   alex|    2019-02-01|   1500|2023-06-01 00:00:00|    null|      true|
+-------+--------------+-------+-------------------+--------+----------+



In [61]:
mergeTable=(dTable_initial.alias("old")
      .merge(df_rows_for_upsert.drop("action").alias("new"),
             condition = "old.account = new.account AND old.dt_transaction = new.dt_transaction AND old.balance = new.balance AND old.is_current = True")
            .whenMatchedUpdate(
                set = {
                "dt_transaction": f.col("old.dt_transaction"),
                "account": f.col("old.account"),
                "balance": f.col("old.balance"),
                "valid_from":f.col("old.valid_from"),
                "valid_to": f.current_timestamp(),
                "is_current": f.lit(False)
            })
            .whenNotMatchedInsertAll()
      .execute()
    )
            
dTable_initial.toDF().sort("account","dt_transaction").show(truncate=False)

+-------+--------------+-------+--------------------+--------------------+----------+
|account|dt_transaction|balance|          valid_from|            valid_to|is_current|
+-------+--------------+-------+--------------------+--------------------+----------+
|   alex|    2019-01-01|   1000| 2023-06-01 00:00:00|                null|      true|
|   alex|    2019-02-01|   1500| 2023-06-01 00:00:00|                null|      true|
|   alex|    2019-03-01|   1700| 2023-06-01 00:00:00|2023-07-21 10:18:...|     false|
|   alex|    2019-03-01|   3300|2023-07-21 10:18:...|                null|      true|
|  maria|    2020-01-01|   5000| 2023-06-01 00:00:00|                null|      true|
|  peter|    2021-01-01|    100|2023-07-21 10:18:...|                null|      true|
+-------+--------------+-------+--------------------+--------------------+----------+



In [62]:
print("++ Vorher ohne deactivierte Zeile und ohne neue Zeilen")
dTable_initial.toDF().show()

print("++ Nach Upsert mit SCD2 Historisierung")
dTable_initial.toDF().sort("account","dt_transaction").show(truncate=False)

++ Vorher ohne deactivierte Zeile und ohne neue Zeilen


+-------+--------------+-------+--------------------+--------------------+----------+
|account|dt_transaction|balance|          valid_from|            valid_to|is_current|
+-------+--------------+-------+--------------------+--------------------+----------+
|   alex|    2019-03-01|   1700| 2023-06-01 00:00:00|2023-07-21 10:18:...|     false|
|   alex|    2019-03-01|   3300|2023-07-21 10:18:...|                null|      true|
|  peter|    2021-01-01|    100|2023-07-21 10:18:...|                null|      true|
|  maria|    2020-01-01|   5000| 2023-06-01 00:00:00|                null|      true|
|   alex|    2019-01-01|   1000| 2023-06-01 00:00:00|                null|      true|
|   alex|    2019-02-01|   1500| 2023-06-01 00:00:00|                null|      true|
+-------+--------------+-------+--------------------+--------------------+----------+

++ Nach Upsert mit SCD2 Historisierung
+-------+--------------+-------+--------------------------+--------------------------+----------+
|a

In [67]:
print("++ aktuell gültiger Rand")
dTable_initial.toDF().where(f.col("is_current")==True).sort("account","dt_transaction").show(truncate=False)

print("++ Veränderung der Daten von account alex am ersten März")
dTable_initial.toDF().where((f.col("account")=="alex") & (f.col("dt_transaction")=="2019-03-01") ).sort("account","valid_from").show(truncate=False)

++ aktuell gültiger Rand
+-------+--------------+-------+--------------------------+--------+----------+
|account|dt_transaction|balance|valid_from                |valid_to|is_current|
+-------+--------------+-------+--------------------------+--------+----------+
|alex   |2019-01-01    |1000   |2023-06-01 00:00:00       |null    |true      |
|alex   |2019-02-01    |1500   |2023-06-01 00:00:00       |null    |true      |
|alex   |2019-03-01    |3300   |2023-07-21 10:18:18.101335|null    |true      |
|maria  |2020-01-01    |5000   |2023-06-01 00:00:00       |null    |true      |
|peter  |2021-01-01    |100    |2023-07-21 10:18:18.101335|null    |true      |
+-------+--------------+-------+--------------------------+--------+----------+

++ Veränderung der Daten von account alex am ersten März


+-------+--------------+-------+--------------------------+--------------------------+----------+
|account|dt_transaction|balance|valid_from                |valid_to                  |is_current|
+-------+--------------+-------+--------------------------+--------------------------+----------+
|alex   |2019-03-01    |1700   |2023-06-01 00:00:00       |2023-07-21 10:18:18.101335|false     |
|alex   |2019-03-01    |3300   |2023-07-21 10:18:18.101335|null                      |true      |
+-------+--------------+-------+--------------------------+--------------------------+----------+



#### Erkenntnisse Delta
* Wie funktioniert das Metadatenmanagement und was steht im Delta Log?
* Schema Evolution: Kann das Schema erweitert werden, also eine neue Spalte angefügt werden?
* Schema Enforcement on write: Kann eine Spalte mit falschem Datetyp einfach beim schreiben hinzugefügt werden? 
* Schema Enforcement on read: Kann ein Verzeichnis mit mehreren Parquet Dateien bei der eine Spalte ein anderes Schema hat gelesen werden? (um die Ecke Denk Frage)
* Was ermöglichen mir die Metadaten (Historie, Audit etc)
* Was ist der Vorteil der Merge Funktion? Wie müsste ich sonst Dateibasiert einen Merge/Update durchführen?


<hr style="height: 3px; background: gray;">

## Iceberg
Das Iceberg Format Ist vergleichbar mit dem Delta Format Parquet, Avro oder ORC Dateien einen zusätzlichen Layer an Metadatan hinzu und erfüllt mit dem entsprechenden Treiber alle ACID Eigenschaften einer Datenbank.   
**Unterschied zu Delta:**  
* Das Delta Format verwendet ein Transaktionsprotokoll, um inkrementelle Updates zu erfassen und eine vollständige Versionierung der Daten zu ermöglichen. Dies bedeutet, dass Delta Änderungen an den Daten in einem einzelnen Parquet-Dateisystem erfasst. Delta ist eng mit Spark und der Delta Lake Plattform von Databricks verbunden und verwedet als darunterliegendes Datenformat immer Parqeut Dateien
* Iceberg hat ebenfalls eine append-only-Architektur, bei der Daten in sogenannten "Snapshot-Dateien" gespeichert werden. Iceberg ist älter und unabhängig von einer speziellen Plattform. Daher es gibt wesentlich mehr Big Data Engines die Iceberg voll unterstützen (Spark, Hive, Trino, Drill, Presto).

**Naming Unterschiede:**  
Bei Delta spricht man von `Versionen` und `Time Travel`. 
Bei Iceberg heißt dies `Snapshots` und `Snapshot roleback`  

Eine weitere Besonderheit bei Iceberg ist, dass es für Daten die Möglichkeiten von Branching und Tagging gibt. Es können also Variationen der Daten in einem z.B. Development Branch gepflegt und bearbeitet weiter entwickelt werden (https://iceberg.apache.org/docs/latest/branching/)

Im Unterchied zu Delta benötigt Spark zum arbeiten mit Iceberg immer einen Catalog oder Metastore um Details über die Dateien abzulegen.

### Iceberg: Catalog und Metastore

In [5]:
# Anzeigen der in Spark eingebundenen Metadaten Cataloge
# Diese Spark Session wurde mit einer Anbindung an den Hive Metastore gestartet
## Konfiguration aus für die Spark Session erzeugt einen Catalog ice vom Typ hive
## conf.set("spark.sql.catalog.spark_catalog", "org.apache.iceberg.spark.SparkSessionCatalog")
## conf.set("spark.sql.catalog.ice","org.apache.iceberg.spark.SparkCatalog") 
## conf.set("spark.sql.catalog.ice.type","hive") 
## conf.set("spark.sql.catalog.ice.uri","thrift://hive-metastore.hive.svc.cluster.local:9083") 


spark.sql("SHOW catalogs").show()
spark.sql("SHOW databases from ice").show()
spark.sql("show tables from ice.iceberg").show()

+-------------+
|      catalog|
+-------------+
|spark_catalog|
+-------------+

+---------+
|namespace|
+---------+
|     data|
|  default|
|   export|
|  iceberg|
+---------+

+---------+---------+-----------+
|namespace|tableName|isTemporary|
+---------+---------+-----------+
|  iceberg|  iceberg|      false|
+---------+---------+-----------+



In [ ]:
# Zunächst muss in dem Ice Catalog eine Datenbankabstraktion angelegt werden mit dem s3 Pfad wo die Daten zu dieser Tabellen liegen
# create a Database(name=<db_name>, locationUri='s3a://<bucket>/')
spark.sql(f"CREATE DATABASE IF NOT EXISTS ice.iceberg LOCATION 's3a://{bucket}/'")

In [68]:
# Jetzt erscheint diese Datenbank im Iceberg Catalog
spark.sql("SHOW databases from ice").show()

+---------+
|namespace|
+---------+
|     data|
|  default|
|   export|
|  iceberg|
+---------+



In [69]:
# Es gibt aber noch keine Tabellen in dieser Datenbankabstraktion
spark.sql("show tables from ice.iceberg").show()

+---------+---------+-----------+
|namespace|tableName|isTemporary|
+---------+---------+-----------+
|  iceberg|  iceberg|      false|
+---------+---------+-----------+



In [ ]:
# Die Tabelle kann ach wieder aus dem Hive Metastore, aus dem Catalog gelöscht werden
# !! Wichtig immer erst die Tabelle UND das Schema aus dem Metastore löschen bevor die Daten in s3 entfernt werden
#spark.sql("drop table iceberg.iceberg")

In [ ]:
# Um Daten unter Verwendung des Catalogs nach s3 zu schreiben wird nicht der direkte Pfad verwendet sondern die Datenbank Referenz wo der Pfad hinterlegt ist
# Die Datenbank iceberg -> s3://bucket/
# die Tabelle soll jetzt nach ->s3://bucket/iceberg
# geschrieben wird mit der Methode `saveAsTable("datenbank.tabelle")` und der Tabellennamen wir dann als Prefix auf s3 verwendet
write_iceberg=(df1
                  .write
                  .format("iceberg")
                  .mode("overwrite")
                  .saveAsTable("ice.iceberg.iceberg")
               )


In [70]:
# Versuche auf dem Dateisystem mit ls und cat die Struktur der Metadaten zu verstehen
# Dieser Artikel kann dabei helfen https://medium.com/snowflake/understanding-iceberg-table-metadata-b1209fbcc7c3
ls(bucket,"iceberg")

iceberg/data/00000-7912-ace0e8c1-fb4a-4812-bdf3-6d4c0ad58a06-00001.parquet
iceberg/data/00000-7920-9dd85db3-402a-4657-856c-f0e8526d3256-00001.parquet
iceberg/data/00001-7913-df5b342a-250c-4439-a721-bf25810804f3-00001.parquet
iceberg/data/00002-7914-112c31b7-0460-4fed-b4c9-8b9a5f1dbfd5-00001.parquet
iceberg/metadata/00000-d4590277-2b88-433d-aa9f-8ba595501aef.metadata.json
iceberg/metadata/00001-fb578df1-af09-4ce8-854a-d66befc4519b.metadata.json
iceberg/metadata/00002-becfce86-8178-4223-8c6d-84baaf06b31b.metadata.json
iceberg/metadata/0a07116f-9521-4d1d-bc4a-3a729535f621-m0.avro
iceberg/metadata/4d380afb-46c9-4d4c-a877-135bf0ec57a9-m0.avro
iceberg/metadata/snap-2832191032484601030-1-0a07116f-9521-4d1d-bc4a-3a729535f621.avro
iceberg/metadata/snap-868766759905674702-1-4d380afb-46c9-4d4c-a877-135bf0ec57a9.avro


In [71]:
# zeige die Daten wieder an
iceberg_df = spark.read.table("ice.iceberg.iceberg")
iceberg_df.show()

# Da es sich bereits um eine Tabellenabstraktion handelt, kann auch direkt mit Spark SQL gelesen werden
spark.sql("SELECT * FROM ice.iceberg.iceberg;").show()

+---+-------+--------------+-------+-------------+
| id|account|dt_transaction|balance|          new|
+---+-------+--------------+-------+-------------+
|  1|   otto|    2019-10-01|   4444|neue Spalte 1|
|  1|   alex|    2019-01-01|   1000|         null|
|  2|   alex|    2019-02-01|   1500|         null|
|  4|  maria|    2020-01-01|   5000|         null|
|  3|   alex|    2019-03-01|   1700|         null|
+---+-------+--------------+-------+-------------+



+---+-------+--------------+-------+-------------+
| id|account|dt_transaction|balance|          new|
+---+-------+--------------+-------+-------------+
|  1|   alex|    2019-01-01|   1000|         null|
|  2|   alex|    2019-02-01|   1500|         null|
|  4|  maria|    2020-01-01|   5000|         null|
|  3|   alex|    2019-03-01|   1700|         null|
|  1|   otto|    2019-10-01|   4444|neue Spalte 1|
+---+-------+--------------+-------+-------------+



### Iceberg: Schema Evolution
**Aufgabe:** Verstehe wie eine Spaltenerweiterung bei Iceberg gemacht wird

In [ ]:
# Füge Datensatz 2 mit einer zusätzlichen Spalte hinzu
write_iceberg=(df3
               .write
               .format("iceberg")
               .option("mergeSchema","true")
               .mode("append")
               .saveAsTable("ice.iceberg.iceberg")
               )

In [74]:
# Um eine neue Spalte hinzuzufügen muss das Schema wie bei einer SQL Tabelle zuerst geändert werden. Es gibt keine automatische Anpassung wie bei Delta via die Option mergeSchema
# Ändere das Schema mit folgendem Befehl und führe dann obige Schreiboperation nochmal durch
spark.sql("ALTER TABLE ice.iceberg.iceberg ADD COLUMNS (new VARCHAR(50))")

In [72]:
spark.sql("SELECT * FROM ice.iceberg.iceberg;").show()

+---+-------+--------------+-------+-------------+
| id|account|dt_transaction|balance|          new|
+---+-------+--------------+-------+-------------+
|  1|   otto|    2019-10-01|   4444|neue Spalte 1|
|  1|   alex|    2019-01-01|   1000|         null|
|  2|   alex|    2019-02-01|   1500|         null|
|  4|  maria|    2020-01-01|   5000|         null|
|  3|   alex|    2019-03-01|   1700|         null|
+---+-------+--------------+-------+-------------+



### Iceberg: Schema Enforcement

**Aufgabe:** verstehe wie Schema Enforcement bei Iceberg funktioniert. Korrigiere den Code bis die Zeile korrekt angefügt werden kann

In [75]:
# füge eine Zeile mit falschen Datetyp für eine bestehenden Spalte an
try:
    write_iceberg=(df2
           # Eine Zeile aus dem df2 filtern
           .where(f.col("account")=="peter")
           # Bestehenden Spaltentyp ändern
           .withColumn("id", f.col("id").cast("string"))
           #.withColumn("new", f.lit("peter").cast("string"))
           .write
           .format("iceberg")
           .mode("append") # append
           .saveAsTable("ice.iceberg.iceberg")
          )
    print("Zeile angefügt")
except Exception as error:
    print("ERROR Writing to Iceberg")
    print(error)

ERROR Writing to Iceberg
Cannot write incompatible data to table 'ice.iceberg.iceberg':
- Cannot safely cast 'id': string to bigint
- Cannot find data for output column 'new'


In Iceberg kann der Typ einer bestehenden Tabelle nicht geändert werden.   
Einzige Möglichkeit ist es die Tabelle einzulesen, neu zu casten und in einen neue Tabelle zu schreiben.   
Diese kann dann anschließend wieder umbenannt werden auf den alten Namen

### Iceberg: History und Metadaten
Iceberg bietet zahlreiche Funktionen um die verschiedenen Metadaten auszulesen.   
Die Funktionen werden immer an die Tabelle angefügt und funktionieren sowohl in der Dataframe als auch in der SQL Syntax
```
spark.read.table("ice.iceberg.iceberg.history").show()
spark.sql("SELECT * FROM ice.iceberg.iceberg.history;").show()
```

**Aufgabe:** Untersuche und verstehe die Metadaten folgender Methoden:  
`history`, `files`, `snapshots`, `manifests`, `partitions`

Weitere Informationen zu den Metadaten und dem Zugrif darauf finden sich in der Dokumentation
https://iceberg.apache.org/docs/latest/spark-queries/

In [83]:
spark.sql("SELECT * FROM ice.iceberg.iceberg.history;").show(truncate=False)

+-----------------------+-------------------+------------------+-------------------+
|made_current_at        |snapshot_id        |parent_id         |is_current_ancestor|
+-----------------------+-------------------+------------------+-------------------+
|2023-07-21 00:04:45.452|868766759905674702 |null              |true               |
|2023-07-21 00:08:50.595|2832191032484601030|868766759905674702|true               |
+-----------------------+-------------------+------------------+-------------------+



In [90]:
spark.sql("SELECT * FROM ice.iceberg.iceberg.snapshots;").select("committed_at","snapshot_id","operation","summary.added-data-files","summary.added-records","summary.total-records","summary.added-data-files").show(truncate=False)

+-----------------------+-------------------+---------+----------------+-------------+-------------+----------------+
|committed_at           |snapshot_id        |operation|added-data-files|added-records|total-records|added-data-files|
+-----------------------+-------------------+---------+----------------+-------------+-------------+----------------+
|2023-07-21 00:04:45.452|868766759905674702 |append   |3               |4            |4            |3               |
|2023-07-21 00:08:50.595|2832191032484601030|append   |1               |1            |5            |1               |
+-----------------------+-------------------+---------+----------------+-------------+-------------+----------------+



In [93]:
iceber_files=spark.sql("SELECT * FROM ice.iceberg.iceberg.files;")

iceber_files.printSchema()
iceber_files.select("file_path","record_count","value_counts","lower_bounds","null_value_counts").show(truncate=False)


root
 |-- content: integer (nullable = true)
 |-- file_path: string (nullable = false)
 |-- file_format: string (nullable = false)
 |-- spec_id: integer (nullable = true)
 |-- record_count: long (nullable = false)
 |-- file_size_in_bytes: long (nullable = false)
 |-- column_sizes: map (nullable = true)
 |    |-- key: integer
 |    |-- value: long (valueContainsNull = false)
 |-- value_counts: map (nullable = true)
 |    |-- key: integer
 |    |-- value: long (valueContainsNull = false)
 |-- null_value_counts: map (nullable = true)
 |    |-- key: integer
 |    |-- value: long (valueContainsNull = false)
 |-- nan_value_counts: map (nullable = true)
 |    |-- key: integer
 |    |-- value: long (valueContainsNull = false)
 |-- lower_bounds: map (nullable = true)
 |    |-- key: integer
 |    |-- value: binary (valueContainsNull = false)
 |-- upper_bounds: map (nullable = true)
 |    |-- key: integer
 |    |-- value: binary (valueContainsNull = false)
 |-- key_metadata: binary (nullable = tr

In [99]:
spark.sql("SELECT * FROM ice.iceberg.iceberg.snapshots;").select("committed_at","snapshot_id","summary.added-data-files","summary.added-records","summary.total-data-files","summary.total-delete-files").show(truncate=False)

+-----------------------+-------------------+----------------+-------------+----------------+------------------+
|committed_at           |snapshot_id        |added-data-files|added-records|total-data-files|total-delete-files|
+-----------------------+-------------------+----------------+-------------+----------------+------------------+
|2023-07-21 00:04:45.452|868766759905674702 |3               |4            |3               |0                 |
|2023-07-21 00:08:50.595|2832191032484601030|1               |1            |4               |0                 |
+-----------------------+-------------------+----------------+-------------+----------------+------------------+



<details style="border: 1px solid #aaa; border-radius: 4px; padding: 0.5em 0.5em 0; background-color:#F5F5F5" class="solution" >
<summary style="margin: -0.5em -0.5em 0; padding: 0.5em;"></summary>
<code>
spark.sql("SELECT * FROM iceberg.iceberg.history;").show()
spark.sql("SELECT * FROM iceberg.iceberg.files;").show()
spark.sql("SELECT * FROM iceberg.iceberg.snapshots;").show()

## alternative syntax example:
spark.read.format("iceberg").load("iceberg.iceberg.files").show()
</code>
</details>

### Iceberg: Iceberg: Time Travel
Bei Iceberg gibt es statt Versionen Snapshots und einige fein granularere Möglichkeiten in der Daten Historie zurück zu gehen.
z.B. können alle Time Travel Operationne auch auf die Metadaten angewendet werden. Es kann also nachgeschaut werden wie die Metadaten zu einem bestimmten Zeitpunkt/Snapshot ausgesehen haben.

- ```snapshot-id``` selects a specific table snapshot
- ```as-of-timestamp``` selects the current snapshot at a timestamp, in milliseconds
- ```branch``` selects the head snapshot of the specified branch. Note that currently branch cannot be combined with as-of-timestamp.
- ```tag``` selects the snapshot associated with the specified tag. Tags cannot be combined with as-of-timestamp.

In [100]:
# from the results of iceberg_table.snapshots get the snapshots IDs
snapshot1 = spark.read \
                 .option("snapshot-id", "2832191032484601030") \
                 .format("iceberg") \
                 .load("ice.iceberg.iceberg").show()

+---+-------+--------------+-------+-------------+
| id|account|dt_transaction|balance|          new|
+---+-------+--------------+-------+-------------+
|  1|   alex|    2019-01-01|   1000|         null|
|  2|   alex|    2019-02-01|   1500|         null|
|  4|  maria|    2020-01-01|   5000|         null|
|  3|   alex|    2019-03-01|   1700|         null|
|  1|   otto|    2019-10-01|   4444|neue Spalte 1|
+---+-------+--------------+-------+-------------+



<hr style="height: 3px; background: gray;">

In [ ]:
spark.stop()